# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [113]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [114]:
cities_data_file = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(cities_data_file)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tocache,-8.18,-76.51,93.76,38,64,1.54,PE,1603900279
1,margate,51.38,1.39,52.00,81,100,1.99,GB,1603900279
2,punta arenas,-53.15,-70.92,48.20,49,75,34.45,CL,1603900279
3,kapaa,22.08,-159.32,76.48,70,8,4.41,US,1603900280
4,beringovskiy,63.05,179.32,25.90,95,41,13.67,RU,1603900280


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [115]:
# Get the locations and Humidity
locations = cities_df[["Lat","Lng"]].astype('float')
humidity = cities_df["Humidity"].astype('float')

gmaps.configure(api_key=g_key)

fig = gmaps.figure(center=(45.5051064,-122.6750261), zoom_level=2)

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=100,
    point_radius = 1.25    
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [116]:
ideal_vacation_weather = cities_df[
    (cities_df["Max Temp"] < 56) & 
    (cities_df["Max Temp"] > 44) &
    (cities_df["Wind Speed"] < 10) &
    (cities_df["Humidity"] < 50) &
    (cities_df["Cloudiness"] < 10)
]
ideal_vacation_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,guadalupe y calvo,26.10,-106.97,55.74,43,1,4.59,MX,1603900297
138,saint george,37.10,-113.58,50.00,33,1,4.70,US,1603900197
144,seoul,37.57,126.98,50.00,39,1,3.36,KR,1603900261
234,kingman,35.19,-114.05,54.00,25,1,3.36,US,1603900366
254,mount isa,-20.73,139.50,55.40,35,0,8.34,AU,1603900376
280,matamoros,25.53,-103.25,50.00,43,5,5.82,MX,1603900386
343,changji,44.02,87.32,46.40,45,0,2.24,CN,1603900408
408,zabol,32.25,67.25,47.80,40,0,8.99,AF,1603900437


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [117]:
hotel_df = ideal_vacation_weather.copy(deep=True).reset_index(drop=True, inplace=False)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,guadalupe y calvo,26.10,-106.97,55.74,43,1,4.59,MX,1603900297,
1,saint george,37.10,-113.58,50.00,33,1,4.70,US,1603900197,
2,seoul,37.57,126.98,50.00,39,1,3.36,KR,1603900261,
3,kingman,35.19,-114.05,54.00,25,1,3.36,US,1603900366,
4,mount isa,-20.73,139.50,55.40,35,0,8.34,AU,1603900376,
5,matamoros,25.53,-103.25,50.00,43,5,5.82,MX,1603900386,
6,changji,44.02,87.32,46.40,45,0,2.24,CN,1603900408,
7,zabol,32.25,67.25,47.80,40,0,8.99,AF,1603900437,


In [118]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row["Lng"]
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params)
    
    hotels = response.json()
    
    #pprint(hotels)
    
    try:
        hotel_name = hotels['results'][0]['name']
    except (KeyError, IndexError):
        hotel_name = "No Hotel Found"
        
    hotel_df.loc[index, "Hotel Name"] = hotel_name
    
hotel_df        

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,guadalupe y calvo,26.10,-106.97,55.74,43,1,4.59,MX,1603900297,"Isabel Hotel & Suites, Guadalupe y Calvo"
1,saint george,37.10,-113.58,50.00,33,1,4.70,US,1603900197,"The Advenire, Autograph Collection"
2,seoul,37.57,126.98,50.00,39,1,3.36,KR,1603900261,K-POP HOTEL Seoul Tower
3,kingman,35.19,-114.05,54.00,25,1,3.36,US,1603900366,Best Western Plus King's Inn & Suites
4,mount isa,-20.73,139.50,55.40,35,0,8.34,AU,1603900376,Isa Hotel
5,matamoros,25.53,-103.25,50.00,43,5,5.82,MX,1603900386,hotel kbisite
6,changji,44.02,87.32,46.40,45,0,2.24,CN,1603900408,Garden Hotel
7,zabol,32.25,67.25,47.80,40,0,8.99,AF,1603900437,No Hotel Found


In [119]:

fig = gmaps.figure(center=(45.5051064,-122.6750261), zoom_level=1)

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=100,
    point_radius = 1.25    
)

fig.add_layer(heat_layer)

In [120]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [121]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    hotel_locations,
    info_box_content=hotel_info
)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))